# Introduction to 'saturation'

This is a small example to introduce the technique of saturation. This is an advanced modelling technique in ASP, that allows you to model [coNP](https://en.wikipedia.org/wiki/Co-NP)-type problems.

Let's start by setting up the basics.

In [1]:
import clingo

def print_answer_sets(program):
    
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print(f"Answer set: {{{', '.join(sorted_model)}}}")
    
    control.configuration.solve.models = 0
    answer = control.solve(on_model=on_model)
    
    if answer.satisfiable == False:
        print("No answer sets")

## Example

To explain the technique of saturation, we'll use the example problem of deciding if a given propositional logic formula in CNF is **un**satisfiable.

The typical approach is to express satisfiability, in the sense that we construct an program *P* whose answer sets correspond to satisfying assignments, and thus there is at least one answer set if and only if the formula is satisfiable.

We will now use saturation to construct a program *P* that has an answer set if and only if the propositional logic formula is **unsatisfiable**.

"What's the point?" you may ask. Indeed, for the simple example of satisfiability we wouldn't need this complicated solution. But there are some more complicated problems that we can only express using this more complicated method (in combination with the typical modelling of NP-type problems).

So let's take a simple example propositional formula $\varphi$ in CNF:

$$ \varphi = (x_1 \vee x_2) \wedge (x_1 \vee \neg x_2) \wedge (\neg x_1 \vee x_2) \wedge (\neg x_1 \vee \neg x_2), $$

and let's encode this formula using some facts.

In [2]:
asp_program = """
    % Declare variables
    var(p(1..2)).

    % Declare formula
    % Clause 1: p(1) OR p(2)
    clause(1,pos(p(1))).
    clause(1,pos(p(2))).

    % Clause 2: ~p(1) OR p(2)
    clause(2,neg(p(1))).
    clause(2,pos(p(2))).

    % Clause 3: p(1) OR ~p(2)
    clause(3,pos(p(1))).
    clause(3,neg(p(2))).

    % Clause 4: ~p(1) OR ~p(2)
    clause(4,neg(p(1))).
    clause(4,neg(p(2))).
    
    % Auxiliary predicate with just clause numbers
    clause(C) :- clause(C,_).
    
    % Auxiliary predicate to negate literals
    negate(pos(V),neg(V)) :- var(V).
    negate(neg(V),pos(V)) :- var(V).
"""

## Saturation in a nutshell

The main ideas behind the technique of saturation are the following.
- We use rules with disjunction in the head to generate a search space that includes all candidate solutions.
- We introduce some rules that check, for everything in this search space, whether it is *incorrect*.
  - Incorrect means that it does not correspond to a solution.
- We add some rules that enforce that for every incorrect option, all atoms involved in this entire process are true.
  - This is what the word 'saturation' refers to: making all involved atoms/facts true.
- Finally, we add a constraint that states that any answer set must be saturated.

This has the effect that there can only be an answer set if there is **no solution**.

If there is no solution (in other words, the search space contains only incorrect options), then the rules in the program enforce saturation. We must select at least one option in the search space, and whichever one we pick will lead to saturation.

If there is some solution (in other words, the search space contains some correct option), then there cannot be an answer set. Why is this? This is because the definition of answer sets says that an answer set must be a *minimal* model of its own reduct. Because of the last constraint, the only candidate for an answer set is the saturated set (i.e., the set with all atoms true). But this is not a minimal model, because there is a smaller model (namely, the one that corresponds to some correct option in the search space).

## Back to our example

Let's run through the steps involved in saturation for our example of unsatisfiability.

The first step is to use some rules with disjunction in the head to generate a search space. Our candidate solutions are the truth assignments to the variables in $\varphi$, so we'll generate a search space that includes this.

In [3]:
asp_program += """
    select(pos(V)) ; select(neg(V)) :- var(V).
"""

Now let's deduce what options in this search space are incorrect. In our case that includes sets that assign some variable both to true and to false.

In [4]:
asp_program += """
    incorrect :- select(pos(V)), select(neg(V)), var(V).
"""

Moreover, the options that do not satisfy some clause are also incorrect. In other words, options that for some clause $c$ set the negation $\neg l$ of each literal $l$ in this clause to true. Let's add a rule for this.

In [5]:
asp_program += """
    incorrect :- clause(I), select(NotL) : clause(I,L), negate(L,NotL).
"""

Now it's time to start the actual saturation. For each incorrect option, we saturate the whole set.

In [6]:
asp_program += """
    saturate :- incorrect.
    select(neg(V)) :- var(V), saturate.
    select(pos(V)) :- var(V), saturate.
    incorrect :- saturate.
"""

And to make sure that only saturated sets remain as answer sets, we add a constraint that enforces this.

In [7]:
asp_program += """
    :- not saturate.
"""

Finally, let's add some `#show` statements to hide the entire saturated set from answer sets.

In [8]:
asp_program += """
    unsat :- saturate.
    #show unsat/0.
"""

In [9]:
print_answer_sets(asp_program)

Answer set: {unsat}


## A satisfiable input

And just to show that this indeed gives answer sets only for unsatisfiable formulas, let's try the whole thing again with a satisfiable formula.

In [10]:
asp_program2 = """
    % Declare variables
    var(p(1..2)).

    % Declare formula
    % Clause 1: p(1) OR p(2)
    clause(1,pos(p(1))).
    clause(1,pos(p(2))).

    % Clause 2: ~p(1) OR p(2)
    clause(2,neg(p(1))).
    clause(2,pos(p(2))).

    % Clause 3: p(1) OR ~p(2)
    clause(3,pos(p(1))).
    clause(3,neg(p(2))).

    % Auxiliary predicate with just clause numbers
    clause(C) :- clause(C,_).
    
    % Auxiliary predicate to negate literals
    negate(pos(V),neg(V)) :- var(V).
    negate(neg(V),pos(V)) :- var(V).
    
    select(pos(V)) ; select(neg(V)) :- var(V).
    
    incorrect :- select(pos(V)), select(neg(V)), var(V).
    incorrect :- clause(I), select(NotL) : clause(I,L), negate(L,NotL).
    
    saturate :- incorrect.
    select(neg(V)) :- var(V), saturate.
    select(pos(V)) :- var(V), saturate.
    incorrect :- saturate.
    
    :- not saturate.
    
    unsat :- saturate.
    #show unsat/0.
"""

print_answer_sets(asp_program2)

No answer sets


Let's consider why there is no answer set for this last program ($P = $ `asp_program2`).
- The only candidate answer set, due to the line `:- not saturate.` is the set that contains `saturate` and therefore (by lines 30–33) all other atoms involved. Let's call this set $A$.
- This set $A$ can only be an answer set (by definition) if it is a (subset-)minimal model of the reduct $P^A$.
- The reduct $P^A$ is almost exactly the same as the program $P$—only line 35 is removed.
- To see why $A$ is not a subset-minimal model of $P^A$, consider the following set $A'$:
  * $A' = \{$ `select(pos(1))`, `select(pos(2))` $\} \cup B'$, where $B'$ contains all statement using `var/1`, `clause/1`, `clause/2`, and `negate/2` needed to satisfy lines 2–23.
- In other words, $A'$ corresponds to the counterexample $\{ x_1 \mapsto 1, x_2 \mapsto 1 \}$.
- Since $A' \subsetneq A$, the only thing remaining to show is that $A'$ satisfies all rules of $P^A$. And this one can do straightforwardly by going over all rules one-by-one. (Try this yourself!)

This also helps to clarify why the set $A$ is an answer set if and only if the formula $\varphi$ were unsatisfiable. If the formula is satisfiable, we can always find such an $A' \subsetneq A$ that witnesses that $A$ is not a subset-minimal model of $P^A$. And if the formula is unsatisfiable, then this is not possible.

## Final notes
Saturation is a tricky and often counterintuitive method, so play around with this and practice with using saturation to get a good feel for it.

This method traces back to a theoretical paper from 1995 (["On the computational cost of disjunctive logic programming: Propositional case"](https://link.springer.com/article/10.1007/BF01536399) by Thomas Eiter and Georg Gottlob). If you find such theoretical work insightful, have a look at (the proof of) Theorem 3.1, where they use the method of saturation.